In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch.utils.data as Data
%matplotlib inline

In [2]:
EPOCH=1
BATCH_SIZE=50
DOWNLOAD_MNIST=False
LR=0.001
train_data=torchvision.datasets.MNIST(
    root='./mnist',
    train=True,
    transform=torchvision.transforms.ToTensor(),#将图片转换为tensor,并进行归一化，（0，1）
    download=DOWNLOAD_MNIST
)
print (train_data.train_data.size())
print(train_data.train_labels.size())
# array=np.arange(625).reshape(25,25)
type(train_data.train_data[2].numpy())
#plt.imshow(train_data.train_data[3].numpy(),cmap='gray')#此函数的功能是把numpy数组转换为图片形式
#plt.title('%i'%train_data.train_labels[1])
#lt.show()

torch.Size([60000, 28, 28])
torch.Size([60000])


numpy.ndarray

In [3]:
a=torch.Tensor([1,2,3])
print (type(a))
a=a.type(torch.IntTensor)
Variable(a)
a[2]

<class 'torch.FloatTensor'>


3

In [4]:
train_loader=Data.DataLoader(dataset=train_data,batch_size=BATCH_SIZE,shuffle=False,num_workers=2)#加载训练集
test_data=torchvision.datasets.MNIST(root='./mnist/',train=False)#此处的train设为false才能运行
test_x=Variable(torch.unsqueeze(test_data.test_data,dim=1),volatile=True).type(torch.FloatTensor)[:5000]/255#使用5000大小的测试集
test_y=test_data.test_labels[:5000]
type(test_y)
print (test_y[1])
#test_y.size()

2


In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1=nn.Sequential(
            nn.Conv2d(1,32,7,1,3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv2=nn.Sequential(
            nn.Conv2d(32,64,5,1,2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.out=nn.Linear(64*7*7,10)
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=x.view(x.size(0),-1)#把batch的维度保留
        output=self.out(x)
        return output
    
cnn=CNN()
print (cnn)


CNN(
  (conv1): Sequential(
    (0): Conv2d (1, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (conv2): Sequential(
    (0): Conv2d (32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (out): Linear(in_features=3136, out_features=10)
)


In [6]:
optimizer=torch.optim.Adam(cnn.parameters(),lr=LR)
loss_func=nn.CrossEntropyLoss()


In [7]:
for epoch in range(EPOCH):
    for step,(x,y) in enumerate(train_loader):
        b_x=Variable(x)#b_x就是50×1×28×28的张量，因为设置的batch大小为50，输入一个通道，每个图片大小28×28
        b_y=Variable(y)#输出的b_y为50的标签，范围为0-9，
        #print (b_y)
        output=cnn(b_x)#output为50×10的向量，因为在网络设置的时候输出的是10个特征,每一个特征值存储的都是概率，
        #print (output.data.numpy())
        #plt.imshow(output.data.numpy())
        #print (output)
        loss=loss_func(output,b_y)#输出为一个数值
        #print (loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step%50==0:
            test_output=cnn(test_x)#将测试集数据输入进行预测，与测试集表标签test_y进行对比
            #print (test_output)
            pred_y=torch.max(test_output,1)[1].data.squeeze()#torch.max(m*n张量，返回每一个m里面最大的那一个值)取最大后验概率,pred_y相当于是m×1维的张量
            #print (pred_y)                                                #之后需要用squeeze压缩成1*m的张量，从而可以比较
            accuracy=sum(pred_y==test_y)/test_y.size(0)
            #print ('EPOCH: ',epoch,'| train loss : %.4f'%loss.data[0],'| test accuracy:%.4f'%accuracy)
    test_output=cnn(test_x[:20])
    pred_y=torch.max(test_output,1)[1].data.numpy().squeeze()
    print (pred_y,'prediction number')
    print (test_y[:20].numpy(),'real number')

Process Process-1:
Traceback (most recent call last):
Process Process-2:
  File "/home/wdd/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/wdd/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wdd/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/wdd/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/home/wdd/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wdd/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/wdd/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  F

KeyboardInterrupt: 

> 注意事项：
* 经过神经网络输出的是variable类型，variable.data是tensor类型
* variable中存储着梯度